# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core


In [ ]:
# | hide
from nbdev.showdoc import *


In [ ]:
# | export

import reader
from typing import *
from bs4 import BeautifulSoup
from rich import print
from rich.panel import Panel
from rich.text import Text
import logging

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, features="lxml")
    return s.text


In [ ]:
# | export

class Feed:
    """RSS feed class 
    """

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)


In [ ]:
# | export

class PyNewsReader:
    def __init__(self, dbpath=None, feeds=List[Feed]):
        if dbpath is None:
            logger.info("Database path not specified, using ./db.sqlite")
            dbpath = "db.sqlite"

        self._reader = reader.make_reader(
            "db.sqlite", plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"]
        )

        self._reader.enable_search()
        self._feed_names = {}

    def _print_entries(self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10):
        """
        Pretty print entries - supports reader.Reader.get_entries arguments
        """
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = Text(
                        "Date: " + e.published.isoformat()[:10], justify="center"
                    )
                else:
                    published_date = Text("Date: Unknown", justify="center")
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = self._get_feed_title(e.original_feed_url)
                print(
                    Panel(
                        published_date
                        + "\n\n"
                        + Text(strip_html(e.summary) + "\n", justify=None),
                        title=f"[link={e.link}]{e.title}[/link]",
                        subtitle=feed_title,
                    )
                )
                print()
            if len(displayed_links) == limit:
                return

    def _search_to_entry(self, search_result):
        for i in self._reader.get_entries():
            if i.id == search_result.id and i.feed_url == search_result.feed_url:
                return i

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed

        Args:
            url (str): URL of pynewsreader feed

        Returns:
            str: Display title
        """
        if url in self._feed_names:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def update(self):
        """Update feeds and search"""
        self._reader.update_feeds()
        self._reader.update_search()

    def _get_entries(self, important: bool = None, read: Union[None, bool] = None, limit: int = 10):
        """Get entries in reader.Entry format

        Args:
            read (reader.Entry.read, optional): Filter on `read` status (None, True, False). Defaults to None.
            limit (int, optional): Number of entries to return. Defaults to 10.

        Returns:
            List[reader.Entry]: List of entries
        """
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def add_feed(self, feed: Feed):
        """Add feed to pynewsreader

        Args:
            feed (Feed): pynewsreader Feed to add
        """
        self._reader.add_feed(feed.url, exist_ok=True)
        if feed.name:
            self._feed_names[feed.url] = feed.name

    def remove_feed(self, feed: Feed):
        """Remove feed from pynewsreader instance

        Args:
            feed (Feed): Feed to remove
        """

        self._reader.delete_feed(feed.url)

    def feeds(self):
        """List pynewsreader feeds

        Returns:
            List[str]: List of names of current pynewsreader feeds
        """
        return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

    def list(self, limit: int = 5, read: bool = None, important: bool = None, mark_as_read: bool = False):
        """Pretty print entries

        Args:
            limit (int, optional): Number of entries to show. Defaults to 5.
            read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
            mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
        """
        self._print_entries(self._get_entries(
            read=read, important=important, limit=limit*2), limit=limit, mark_as_read=mark_as_read)

    def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
        """Search entries and pretty print results

        Args:
            query (str): Search query
            mark_as_read (bool, optional): Mark results as read? Defaults to True.
        """
        self._print_entries([self._search_to_entry(i) for i in self._reader.search_entries(
            query)], mark_as_read=mark_as_read, limit=limit)

    def mark_important(self, entry: reader.Entry):
        """Mark entry as important

        Args:
            entry (reader.Entry): Entry to mark as important
        """
        reader.Reader.mark_entry_as_important(entry)

    def mark_unimportant(self, entry: reader.Entry):
        """Mark entry as important

        Args:
            entry (reader.Entry): Entry to mark as important
        """
        reader.Reader.mark_entry_as_unimportant(entry)

        
    def tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Tag an entry

        Args:
            entry (reader.Entry): Entry to tag
            tag_key (str): Key of tag
            tag_value (Dict, optional): Value of tag. Defaults to None.
        """
        reader.Reader.set_tag(entry, tag_key, tag_value)


In [ ]:
r = PyNewsReader()


## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url='https://ricochet.media/en/feed', name="Richochet Media"),
    Feed(url='https://thetyee.ca/rss2.xml', name="The Tyee"),
    Feed(url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
         name="Toronto Star | Top Stories"),
    Feed(url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
         name="Toronto Star | Investigations"),
    Feed(url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
         name="Toronto Star | Editorials"),
    Feed(url="https://www.macleans.ca/feed/",
         name="Macleans")
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://thenarwhal.ca/feed/",
]:
    r.add_feed(Feed(i))


## List Feeds

In [ ]:
r.feeds()


['Richochet Media',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans',
 'The Narwhal',
 'The Tyee',
 'Toronto Star | Investigations',
 'Toronto Star | Editorials',
 'Toronto Star | Top Stories']

## Update Feeds

In [ ]:
r.update()


## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.list(read=False, limit=5, mark_as_read=True)


╭────────────────────────── ]8;id=317934;https://www.cbc.ca/news/science/spacex-ispace-moon-rover-launch-1.6681992?cmp=rss\Japan's ispace launches world's 1st commercial lunar lander]8;;\ ──────────────────────────╮
│                                                Date: 2022-12-11                                                 │
│                                                                                                                 │
│  A Japanese space startup launched a spacecraft to the moon on Sunday after several delays, a step toward what  │
│                           would be a first for the country and for a private company.                           │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭───────────────────────────── ]8;id=502574;https://www.thestar.com/opinion/editorials/2022/12/11/ottawa-needs-to-end-its-roadblocks-to-helping-afghans.html\Ottawa needs to end its roadblocks to helping Afghans]8;;\ ─────────────────────────────╮
│                                                Date: 2022-12-11                                                 │
│                                                                                                                 │
│     We need to find a way to get to get humanitarian aid to Afghans before the food crisis there get worse.     │
│                                                                                                                 │
╰─────────────────────────────────────────── Toronto Star | Editorials ───────────────────────────────────────────╯

╭───────────────── ]8;id=371229;https://thenarwhal.ca/jane-goodall-hope/\Jane Goodall on hope, fatigue and finding pockets of nature wherever you are]8;;\ ──────────────────╮
│                                                Date: 2022-12-10                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                             By                                                  │
│                                                 Emma Gilchrist                                                  │
│                                                                                                                 │
│                                                                                                                 │
│   At 88 years old, the world’s best-known naturalist is calling on her fans to roll up their sleeves and ‘do    │
│                                                   something’                                                    │
│                                                                                                                 │
╰────────────────────────────────────────────────── The Narwhal ──────────────────────────────────────────────────╯

╭───────────────────────────────────────── ]8;id=658394;https://www.thestar.com/opinion/editorials/2022/12/10/profits-in-the-grocery-aisle.html\Profits in the grocery aisle]8;;\ ──────────────────────────────────────────╮
│                                                Date: 2022-12-10                                                 │
│                                                                                                                 │
│  Grocery chains insist they are not unduly profiting from inflation. Price-pinched shoppers, quite naturally,   │
│                                       would like to be convinced of that.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────── Toronto Star | Editorials ───────────────────────────────────────────╯

╭────────────────────── ]8;id=451360;https://www.cbc.ca/news/business/chatgpt-artificial-intelligence-1.6681401?cmp=rss\Can the new AI tool ChatGPT replace human work? Judge for yourself]8;;\ ───────────────────────╮
│                                                Date: 2022-12-10                                                 │
│                                                                                                                 │
│   A new artificial intelligence tool using natural language processing has captured the public's imagination,   │
│                                       amassing more than a million users.                                       │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

## Search Entries

In [ ]:
r.search("fusion")


╭─────────────────────────── ]8;id=24007;https://www.cbc.ca/news/science/us-fusion-announcement-1.6682497?cmp=rss\Fusion energy 'breakthrough' revealed by U.S. scientists]8;;\ ────────────────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│      U.S. Energy Secretary Jennifer Granholm announced a "major scientific breakthrough" on Tuesday in the      │
│                 decades-long quest to harness fusion, the energy that powers the sun and stars.                 │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭───────────────────────────── ]8;id=705691;https://www.cbc.ca/news/science/nuclear-fusion-explainer-1.6684298?cmp=rss\What is nuclear fusion and why is it such a big deal?]8;;\ ─────────────────────────────╮
│                                                Date: 2022-12-13                                                 │
│                                                                                                                 │
│  The U.S. Department of Energy on Tuesday announced a breakthrough in nuclear fusion, a way of producing clean  │
│ energy that scientists have been working on since the 1940s. So what exactly is it, why is it so important and  │
│                                        what does it mean for the future?                                        │
│                                                                                                                 │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭────────────── ]8;id=732462;https://www.cbc.ca/news/science/what-on-earth-cannabis-plastic-waste-1.6687271?cmp=rss\The cannabis industry has a plastic waste problem — but some are finding solutions]8;;\ ───────────────╮
│                                                Date: 2022-12-15                                                 │
│                                                                                                                 │
│    In this week's issue of our environment newsletter, we look at the plastic waste produced in the cannabis    │
│      industry and what this week's announcement about nuclear fusion means for the future of clean energy.      │
│                                                                                                                 │
╰─────────────────────────────────────────────── CBC | Canada News ───────────────────────────────────────────────╯

In [ ]:
# | hide

# To Do:
# * Command line interface
# * User interface
# * Tags


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
